# Namaph-Sim Model
- u ... physical qty / vector
- D ... diffusion coef / vector
- r ... interaction coef / matrix
- k ... flow io coef / vector
- f(r, u) ... reaction fn
- f(u) ... flow io fn

$$
\begin{align*}
\begin{aligned}
u &= (u_1, u_2, ..., u_n)\\
d &= (d_1, d_2, ..., d_n) \\
r &= \begin{pmatrix}\\
&r_{11},  &\cdots &r_{1n} \\
&\vdots  &\ddots &\vdots \\
&r_{n1},  &\cdots &r_{nn} \\
\end{pmatrix} \\
k &= (k_1, k_2, ..., k_n) \\
\frac{\partial u_i}{\partial t} &= d_i \Delta u_i + f(r, u) + g(k, u)\\
\end{aligned}
\end{align*}
$$

the most simplest $f(r, u)$ case is no-interaction, linear model
$$ \frac{\partial u}{\partial t} = d \cdot \Delta u + r_{(i, \cdot)} \cdot u - (1-k) \cdot u$$

other examples
- Gray Scott (non-linear, 2var simulation)
  - $\frac{\partial u}{\partial t} = D_u \Delta u - uv^2 + F(1-u)$
  - $\frac{\partial v}{\partial t} = D_v \Delta v + uv^2 - (F+k)v$
- Lotka Volterra (non-linear, 2var simulation)
  - $\frac{dN_1}{dt} = N_1(r_1 - \gamma_{12} N_2 - \beta_1N_1) = (r_1N_1 - \gamma_{12} N_1N_2 - \beta_1N_1^2)$
  - $\frac{dN_2}{dt} = N_2(r_2 - \gamma_{21}N_1  - \beta_2N_2) = (r_2N_2 - \gamma_{21}N_1N_2 - \beta_2N_2^2)$

laplacian is given by 

$$\Delta u_{t}^{(x,y)} = \frac{1}{(\Delta x)^2} (u_{t}^{(x,y-1)} + u_{t}^{(x-1,y)} -4u_{t}^{(x,y)} + u_{t}^{(x+1,y)} + u_{t}^{(x,y+1)})$$

and discretization of this model is 

$$u_{t+1}^{(x,y)} = u_{t}^{(x,y)} + \Delta t \lbrace d\cdot \Delta u_t^{(x,y)} - f(r, u_t^{(x,y)}) + g(k, u_t^{(x,y)}) \rbrace$$

FTCS(forward in time centered in space) scheme

In [1]:
from dataclasses import dataclass
from enum import Enum
from typing import Any

import numpy as np
import numpy.typing as npt
from tqdm.notebook import tqdm

In [2]:
Grid = npt.NDArray[float]

class LaplacianMode(Enum):
    neumann = "neumann"
    moore = "moore"

class BoundaryCondition(Enum):
    periodic = "periodic"
    dirichlet = "dirichlet"
    neumann = "neumann"

class History():
    name: list[str]
    def __init__(self, *args) -> None:
        self.name = args
        for arg in args:
            setattr(self, arg, [])
    
    def append(self, **kwargs) -> None:
        for k, v in kwargs.items():
            getattr(self, k).append(v)
    
    def __repr__(self) -> str:
        return f"History({self.name})"
        
    
class NamaphSim2D():
    dt: float
    dx: float
    
    def __init__(self, dt: float, dx: float) -> None:
        self.dt = dt
        self.dx = dx
    
    def init_boundary(
        self,
        field: Grid,
        mode: BoundaryCondition=BoundaryCondition.periodic,
        **option: dict[str, int|float|npt.NDArray]
    ) -> Grid:
        x, y = field.shape
        new_field = np.zeros((x+2, y+2))
        new_field[1:-1, 1:-1] = field
        return self.update_boundary(new_field, mode, **option)
    
    def laplacian(
        self,
        field: Grid,
        mode: LaplacianMode=LaplacianMode.moore
    ) -> Grid:
        mu = self.dx**2
        match mode:
            case LaplacianMode.neumann:
                val = mu * self.neumann_nbhd(field)
                return np.pad(val, [(1,), (1,)], "constant")
            case LaplacianMode.moore:
                val = mu * self.moore_nbhd(field)
                return np.pad(val, [(1,), (1,)], "constant")
    
    def update_boundary(
        self,
        field: Grid,
        mode: BoundaryCondition=BoundaryCondition.periodic,
        **option: dict[str, int|float|npt.NDArray]
    ):
        match mode:
            case BoundaryCondition.periodic:
                return self.periodic_bc(field)
            case BoundaryCondition.dirichlet:
                return self.dirichlet_bc(field, option['value'])
            case BoundaryCondition.neumann:
                return self.neumann_bc(field)
    
    def neumann_nbhd(self, u: Grid) -> Grid:
        ret =  (                  u[ :-2, 1:-1] +
                 u[1:-1, :-2] - 4*u[1:-1, 1:-1] + u[1:-1, 2:]
                              +   u[2:  , 1:-1]               )
        return ret
    
    def moore_nbhd(self, u: Grid) -> Grid:
        ret =  ( u[ :-2, :-2] +   u[ :-2, 1:-1] + u[ :-2, 2:] +
                 u[1:-1, :-2] - 8*u[1:-1, 1:-1] + u[1:-1, 2:] +
                 u[2:  , :-2] +   u[2:  , 1:-1] + u[2:  , 2:] )
        return ret
    
    def periodic_bc(self, u: Grid) -> Grid:
        # Check: values in the corner would be assigned twice.
        u[[0,-1], :     ] = u[[-2,1], :     ]
        u[:     , [0,-1]] = u[:     , [-2,1]]
        return u
    
    def dirichlet_bc(self, u:Grid, value:npt.NDArray) -> Grid:
        u[[0,-1], :     ] = value[0], value[1]
        u[:     , [0,-1]] = value[2], value[3]
        return u
    
    def neumann_bc(self, u:Grid) -> Grid:
        u[[0,-1], :     ] = u[[1,-2], :     ]
        u[:     , [0,-1]] = u[:     , [1,-2]]
        return u

In [3]:
@dataclass
class GrayScottParams:
    dt: float
    dx: float
    
    DiffusionRateF: float
    DiffusionRateK: float
    
    FeedRate: float
    KillRate: float
    
class DiffReact(NamaphSim2D):
    Df: float
    Dk: float
    F: float
    K: float
    
    def __init__(self, params: GrayScottParams) -> None:
        super().__init__(params.dt, params.dx)
        self.Df = params.DiffusionRateF
        self.Dk = params.DiffusionRateK
        self.F = params.FeedRate
        self.K = params.KillRate
    
    def diffuse(self, feeder: Grid, killer: Grid, mode: LaplacianMode) -> tuple[Grid, Grid]:
        return self.Df * self.laplacian(feeder, mode), self.Dk * self.laplacian(killer, mode)
    
    def react(self, feeder: Grid, killer: Grid) -> tuple[Grid, Grid]:
        value = feeder * (killer ** 2)
        return -value, +value
    
    def flow(self, feeder: Grid, killer: Grid) -> tuple[Grid, Grid]:
        return self.F * (1 - feeder), -(self.F + self.K) * killer
    
    def step(self, feeder: Grid, killer: Grid, mode:LaplacianMode) -> tuple[Grid, Grid]:
        df, dk = self.diffuse(feeder, killer, mode)
        rf, rk = self.react(feeder, killer)
        ff, fk = self.flow(feeder, killer)
        return self.update_boundary(feeder+self.dt*(df+rf+ff)), self.update_boundary(killer+self.dt*(dk+rk+fk))
    
    def run(
        self,
        feeder: Grid,
        killer: Grid,
        n_iter: int = 100,
        n_per_record: int = 10
    ) -> tuple[Grid, Grid]:
        f, k = self.init_boundary(feeder), self.init_boundary(killer)
        gif = lambda k: np.uint8(255*(k-k.min()) / (k.max()-k.min()))
        
        hist = History('feeder', 'killer', 'gif')
        hist.append(feeder = f, killer = k, gif = gif(k))
                
        f, k = feeder, killer
        
        for t in tqdm(range(n_iter)):
            f, k = self.step(f, k, LaplacianMode.moore)
            hist.append(feeder = f, killer = k)
            if t % n_per_record == 0:
                hist.append(gif = gif(k))
        return hist

In [4]:
def init(n):
    u = np.ones((n+2,n+2))
    v = np.zeros((n+2,n+2))
    
    x, y = np.meshgrid(np.linspace(0, 1, n+2), np.linspace(0, 1, n+2))

    mask = (0.4<x) & (x<0.6) & (0.4<y) & (y<0.6)
    
    u[mask] = 0.50
    v[mask] = 0.25
        
    return u, v

In [5]:
params = GrayScottParams(
    dt = 1,
    dx = 1,
    
    DiffusionRateF = .1,
    DiffusionRateK = .05,
    
    FeedRate = 0.0545,
    KillRate = 0.062,
)
feeder, killer = init(300)

model = DiffReact(params)
result = model.run(feeder, killer, n_iter=4000)

  0%|          | 0/4000 [00:00<?, ?it/s]

In [6]:
from PIL import Image
from ipywidgets import interact, IntSlider
import imageio

imageio.mimsave('./output/grayscott.gif', result.gif, format='gif', fps=60)
    
interact(
    lambda iframe: Image.fromarray(result.gif[iframe]), 
    iframe = IntSlider(
        min=0,
        max=len(result.gif)-1,
        step=1,
        value=0, 
        continuous_update=True)
)

interactive(children=(IntSlider(value=0, description='iframe', max=400), Output()), _dom_classes=('widget-inte…

<function __main__.<lambda>(iframe)>